<a href="https://colab.research.google.com/github/karenbennis/Xy/blob/ml_model/pyspark_pipeline_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br>**Connect to Database**<br><br>

In [1]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

#Interact with SQL
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

# Start Spark Session(Creating spark application with name defined by appName()) ---IMPORTED WITH EVERY COLAB NOTEBOOK
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("database_transformation").config("spark.driver.memory","8g").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


--2020-07-24 00:25:21--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.7’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2020-07-24 00:25:21 (9.00 MB/s) - ‘postgresql-42.2.9.jar.7’ saved [914037/914037]



In [2]:
# gcloud login and check the DB
!gcloud auth login
!gcloud config set project 'xy-yelp'
!gcloud sql instances describe 'xy-yelp'

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&code_challenge=TGHQzzth5e5AMSwqAw6CpiH3wec9voSu-hui-MK-he0&code_challenge_method=S256&access_type=offline&response_type=code&prompt=select_account


Enter verification code: 4/2QHe7YUZLypxZoQQ6PhcY2igPX1F0aY8znLdi8Pv6YIxOPyVrLPM_vU

You are now logged in as [helenly25@gmail.com].
Your current project is [xy-yelp].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID
Updated property [core/project].
backendType: SECOND_GEN
connectionName: xy-yelp:northamerica-northeast1:xy-yelp
databaseV

In [3]:
# download and initialize the psql proxy
!wget https://dl.google.com/cloudsql/cloud_sql_proxy.linux.amd64 -O cloud_sql_proxy
!chmod +x cloud_sql_proxy
# "connectionName" is from the previous block
!nohup ./cloud_sql_proxy -instances="xy-yelp:northamerica-northeast1:xy-yelp"=tcp:5432 &
!sleep 30s

cloud_sql_proxy: Text file busy
nohup: appending output to 'nohup.out'


In [4]:
db_password = 'kjhbyelpdb'

In [5]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://127.0.0.1:5432/xy_yelp_db"
config = {"user":"postgres", 
          "password": db_password, 
          "driver":"org.postgresql.Driver"}

**Extract tables**

In [6]:
# Pull review table
review_df2 = spark.read.jdbc(url=jdbc_url, table='review',properties=config)
review_df2.show(5)

+--------------------+--------------------+-----+----+------+-----+-----------+-----------+
|           review_id|         review_text|stars|cool|useful|funny|review_date|review_type|
+--------------------+--------------------+-----+----+------+-----+-----------+-----------+
|cALYebKb5hygdKHql...|This is a very in...|    4|   0|     0|    0| 2011-01-12|     review|
|SawdMXLYD5ytRmMFv...|I LOVE Chic Nails...|    5|   0|     2|    0| 2011-01-20|     review|
|j-jMQdELr6AFkCcEH...|After the Padres ...|    5|   0|     0|    0| 2011-01-06|     review|
|SmUMyCUNrT9HEo_DX...|I have to admit t...|    4|   0|     1|    0| 2010-01-17|     review|
|oTB_mpCKcu-8wayQQ...|Best food, super ...|    5|   0|     1|    0| 2011-01-14|     review|
+--------------------+--------------------+-----+----+------+-----+-----------+-----------+
only showing top 5 rows



In [7]:
# Pull business table
business_df2 = spark.read.jdbc(url=jdbc_url, table='business',properties=config)
business_df2.show(5)

+--------------------+--------------------+
|           review_id|         business_id|
+--------------------+--------------------+
|fWKvX83p0-ka4JS3d...|9yKzy9PApeiPPOUJE...|
|IjZ33sJrzXqU-0X6U...|ZRJwVLyzEJq1VAihD...|
|IESLBzqUCLdSzSqm0...|6oRAC4uyJCsJl1X0W...|
|G-WvGaISbqqaMHlNn...|_1QQZuf4zZOyFCvXc...|
|1uJFq2r5QfJG_6ExM...|6ozycU1RpktNG2-1B...|
+--------------------+--------------------+
only showing top 5 rows



In [8]:
# Pull yelp_user table
user_df2 = spark.read.jdbc(url=jdbc_url, table='yelp_user',properties=config)
user_df2.show(5)

+--------------------+--------------------+
|           review_id|             user_id|
+--------------------+--------------------+
|GJGUHAAONtBSBj53c...|Z3c7xGRfeV-uMkSbA...|
|nQH2KAvAeOJOYKX99...|ryjqXdp68i2I9JPOp...|
|-yKcbjWSlmKC1zTMT...|5W-ruHmpkwLyI6Lla...|
|20aES_-g5Vyqfzojn...|vhxFLqRok6r-D_aQz...|
|W_d9w7yr3koSUXHco...|aBnKTxZzdhabTXfzt...|
+--------------------+--------------------+
only showing top 5 rows



In [9]:
# Join tables
spark_df = review_df2.join(business_df2, on="review_id", how="inner")
spark_df = spark_df.join(user_df2, on="review_id", how="inner")
spark_df.show(5)

+--------------------+--------------------+-----+----+------+-----+-----------+-----------+--------------------+--------------------+
|           review_id|         review_text|stars|cool|useful|funny|review_date|review_type|         business_id|             user_id|
+--------------------+--------------------+-----+----+------+-----+-----------+-----------+--------------------+--------------------+
|-7yxrdY13ay15rGB7...|I have been going...|    5|   0|     0|    0| 2010-01-16|     review|Lh9nz0KYyzE-YRbKu...|ayKW9eWwGFcrtJaHc...|
|-Be0UUGYuiDJVAM_Y...|Since Im big into...|    4|   0|     2|    2| 2011-01-25|     review|pa6K7DGByxBXxcVJ5...|_4lqpCYCqOQzbB6xQ...|
|-nQHHXi-d_yuW301_...|A pleasant place ...|    2|   0|     0|    0| 2011-01-12|     review|GIGI8bJfN6HyPzmEW...|4QORbyhfN01oKR_Gg...|
|2L30O7G8IQ6HILpR0...|part of a social ...|    5|   0|     0|    0| 2010-01-24|     review|qiwajZigq_2twTmYo...|ST8Yzlk2MqKlcaLqL...|
|4x5yLG7_yGLuN-w6f...|I love every plac...|    4|   0|     1| 

**Transformation**

In [10]:
import pyspark.sql.functions as F

spark_df=spark_df.withColumn('length',F.length('review_text'))

In [11]:
spark_df = spark_df.filter((spark_df.stars != 2) & (spark_df.stars != 4))
spark_df.show()

+--------------------+--------------------+-----+----+------+-----+-----------+-----------+--------------------+--------------------+------+
|           review_id|         review_text|stars|cool|useful|funny|review_date|review_type|         business_id|             user_id|length|
+--------------------+--------------------+-----+----+------+-----+-----------+-----------+--------------------+--------------------+------+
|-7yxrdY13ay15rGB7...|I have been going...|    5|   0|     0|    0| 2010-01-16|     review|Lh9nz0KYyzE-YRbKu...|ayKW9eWwGFcrtJaHc...|   670|
|2L30O7G8IQ6HILpR0...|part of a social ...|    5|   0|     0|    0| 2010-01-24|     review|qiwajZigq_2twTmYo...|ST8Yzlk2MqKlcaLqL...|   415|
|5h0EVAee-RDbbKfhd...|A great value for...|    5|   2|     1|    0| 2012-01-21|     review|4VzaYvZntWBRbr8Vm...|feQpvbp8jGBWMuG5u...|   284|
|6jV77Bs_Vu_rHkdUx...|This review is fo...|    3|   1|     2|    1| 2011-01-23|     review|kwq3bK7BzPKLwXKqV...|RwVaQNP1Ag-Seu3U9...|  1092|
|8uV26l7-ktb4

In [12]:
# spark_df=spark_df.withColumn('class',F.when( (spark_df["stars"]==1), 0).when((spark_df["stars"]))
# spark_df.show()

In [13]:
# convert to pandas
pandas_df = spark_df.toPandas()

# Set index
# pandas_df = pandas_df.set_index('review_id')

pandas_df.head()

,review_id,review_text,stars,cool,useful,funny,review_date,review_type,business_id,user_id,length
0,-7yxrdY13ay15rGB7WibMA,I have been going to Arizona Auto Care since a...,5,0,0,0,2010-01-16,review,Lh9nz0KYyzE-YRbKuCYeUw,ayKW9eWwGFcrtJaHcwZUCw,670
1,2L30O7G8IQ6HILpR0t5RFA,"part of a social event, we only had app's here...",5,0,0,0,2010-01-24,review,qiwajZigq_2twTmYofPmDQ,ST8Yzlk2MqKlcaLqL2djBg,415
2,5h0EVAee-RDbbKfhd6FB0g,A great value for a far superior hair cut than...,5,2,1,0,2012-01-21,review,4VzaYvZntWBRbr8VmwCiWg,feQpvbp8jGBWMuG5uqbk3Q,284
3,6jV77Bs_Vu_rHkdUxu9JLQ,This review is for the JCPenny portrait studio...,3,1,2,1,2011-01-23,review,kwq3bK7BzPKLwXKqVRztHg,RwVaQNP1Ag-Seu3U9quGAg,1092
4,8uV26l7-ktb4tZ5gRsIWWg,"The new Harkins Cine Capri, one of the first t...",5,0,1,0,2007-01-23,review,1pGC-0jvQ6vN5rzmCHjmfg,nWouNfZD3Pw08RYizxkqcA,355


In [14]:
# Import dependencies for nltk
# https://towardsdatascience.com/natural-language-processing-nlp-for-machine-learning-d44498845d5b
import nltk

In [15]:
# Import string and punctuations
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [16]:
# Function to remove Punctuation
def remove_punct(text):

  # Discard all punctuations
  text_nopunct = ''.join([char for char in text if char not in string.punctuation])
  return text_nopunct

pandas_df['body_text_clean'] = pandas_df['review_text'].apply(lambda x: remove_punct(x))

pandas_df.head()

,review_id,review_text,stars,cool,useful,funny,review_date,review_type,business_id,user_id,length,body_text_clean
0,-7yxrdY13ay15rGB7WibMA,I have been going to Arizona Auto Care since a...,5,0,0,0,2010-01-16,review,Lh9nz0KYyzE-YRbKuCYeUw,ayKW9eWwGFcrtJaHcwZUCw,670,I have been going to Arizona Auto Care since a...
1,2L30O7G8IQ6HILpR0t5RFA,"part of a social event, we only had app's here...",5,0,0,0,2010-01-24,review,qiwajZigq_2twTmYofPmDQ,ST8Yzlk2MqKlcaLqL2djBg,415,part of a social event we only had apps here q...
2,5h0EVAee-RDbbKfhd6FB0g,A great value for a far superior hair cut than...,5,2,1,0,2012-01-21,review,4VzaYvZntWBRbr8VmwCiWg,feQpvbp8jGBWMuG5uqbk3Q,284,A great value for a far superior hair cut than...
3,6jV77Bs_Vu_rHkdUxu9JLQ,This review is for the JCPenny portrait studio...,3,1,2,1,2011-01-23,review,kwq3bK7BzPKLwXKqVRztHg,RwVaQNP1Ag-Seu3U9quGAg,1092,This review is for the JCPenny portrait studio...
4,8uV26l7-ktb4tZ5gRsIWWg,"The new Harkins Cine Capri, one of the first t...",5,0,1,0,2007-01-23,review,1pGC-0jvQ6vN5rzmCHjmfg,nWouNfZD3Pw08RYizxkqcA,355,The new Harkins Cine Capri one of the first th...


In [17]:
# Tokenization
import re

# Function to Tokenize words
def tokenize(text):

  # W+ means that either a word character (A-Za-z0-9) or a dash (-) can go there
  tokens = re.split('\W+', text)
  return tokens

# Convert to lowercase as Python is case-sensitive
pandas_df['body_text_tokenized'] = pandas_df['body_text_clean'].apply(lambda x: tokenize(x.lower()))

pandas_df.head()

,review_id,review_text,stars,cool,useful,funny,review_date,review_type,business_id,user_id,length,body_text_clean,body_text_tokenized
0,-7yxrdY13ay15rGB7WibMA,I have been going to Arizona Auto Care since a...,5,0,0,0,2010-01-16,review,Lh9nz0KYyzE-YRbKuCYeUw,ayKW9eWwGFcrtJaHcwZUCw,670,I have been going to Arizona Auto Care since a...,"[i, have, been, going, to, arizona, auto, care..."
1,2L30O7G8IQ6HILpR0t5RFA,"part of a social event, we only had app's here...",5,0,0,0,2010-01-24,review,qiwajZigq_2twTmYofPmDQ,ST8Yzlk2MqKlcaLqL2djBg,415,part of a social event we only had apps here q...,"[part, of, a, social, event, we, only, had, ap..."
2,5h0EVAee-RDbbKfhd6FB0g,A great value for a far superior hair cut than...,5,2,1,0,2012-01-21,review,4VzaYvZntWBRbr8VmwCiWg,feQpvbp8jGBWMuG5uqbk3Q,284,A great value for a far superior hair cut than...,"[a, great, value, for, a, far, superior, hair,..."
3,6jV77Bs_Vu_rHkdUxu9JLQ,This review is for the JCPenny portrait studio...,3,1,2,1,2011-01-23,review,kwq3bK7BzPKLwXKqVRztHg,RwVaQNP1Ag-Seu3U9quGAg,1092,This review is for the JCPenny portrait studio...,"[this, review, is, for, the, jcpenny, portrait..."
4,8uV26l7-ktb4tZ5gRsIWWg,"The new Harkins Cine Capri, one of the first t...",5,0,1,0,2007-01-23,review,1pGC-0jvQ6vN5rzmCHjmfg,nWouNfZD3Pw08RYizxkqcA,355,The new Harkins Cine Capri one of the first th...,"[the, new, harkins, cine, capri, one, of, the,..."


In [18]:
# Remove all English stopwords
import nltk
nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('english') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
# Function to remove stopwords
def remove_stopwords(tokenized_list):

  # Remove all stopwords
  text = [word for word in tokenized_list if word not in stopword]
  return text

pandas_df['body_text_nostop'] = pandas_df['body_text_tokenized'].apply(lambda x: remove_stopwords(x))

pandas_df.head()

,review_id,review_text,stars,cool,useful,funny,review_date,review_type,business_id,user_id,length,body_text_clean,body_text_tokenized,body_text_nostop
0,-7yxrdY13ay15rGB7WibMA,I have been going to Arizona Auto Care since a...,5,0,0,0,2010-01-16,review,Lh9nz0KYyzE-YRbKuCYeUw,ayKW9eWwGFcrtJaHcwZUCw,670,I have been going to Arizona Auto Care since a...,"[i, have, been, going, to, arizona, auto, care...","[going, arizona, auto, care, since, 2002, work..."
1,2L30O7G8IQ6HILpR0t5RFA,"part of a social event, we only had app's here...",5,0,0,0,2010-01-24,review,qiwajZigq_2twTmYofPmDQ,ST8Yzlk2MqKlcaLqL2djBg,415,part of a social event we only had apps here q...,"[part, of, a, social, event, we, only, had, ap...","[part, social, event, apps, quite, delicious, ..."
2,5h0EVAee-RDbbKfhd6FB0g,A great value for a far superior hair cut than...,5,2,1,0,2012-01-21,review,4VzaYvZntWBRbr8VmwCiWg,feQpvbp8jGBWMuG5uqbk3Q,284,A great value for a far superior hair cut than...,"[a, great, value, for, a, far, superior, hair,...","[great, value, far, superior, hair, cut, likel..."
3,6jV77Bs_Vu_rHkdUxu9JLQ,This review is for the JCPenny portrait studio...,3,1,2,1,2011-01-23,review,kwq3bK7BzPKLwXKqVRztHg,RwVaQNP1Ag-Seu3U9quGAg,1092,This review is for the JCPenny portrait studio...,"[this, review, is, for, the, jcpenny, portrait...","[review, jcpenny, portrait, studio, responsibl..."
4,8uV26l7-ktb4tZ5gRsIWWg,"The new Harkins Cine Capri, one of the first t...",5,0,1,0,2007-01-23,review,1pGC-0jvQ6vN5rzmCHjmfg,nWouNfZD3Pw08RYizxkqcA,355,The new Harkins Cine Capri one of the first th...,"[the, new, harkins, cine, capri, one, of, the,...","[new, harkins, cine, capri, one, first, things..."


In [20]:
# Stemming
from nltk.stem import PorterStemmer

# Create an instance for stemmer
ps = nltk.PorterStemmer()

# Function for stemming
def stemming(tokenized_text):

  text = [ps.stem(word) for word in tokenized_text]
  return text

pandas_df['body_text_stemmed'] = pandas_df['body_text_nostop'].apply(lambda x: stemming(x))

pandas_df.head()

,review_id,review_text,stars,cool,useful,funny,review_date,review_type,business_id,user_id,length,body_text_clean,body_text_tokenized,body_text_nostop,body_text_stemmed
0,-7yxrdY13ay15rGB7WibMA,I have been going to Arizona Auto Care since a...,5,0,0,0,2010-01-16,review,Lh9nz0KYyzE-YRbKuCYeUw,ayKW9eWwGFcrtJaHcwZUCw,670,I have been going to Arizona Auto Care since a...,"[i, have, been, going, to, arizona, auto, care...","[going, arizona, auto, care, since, 2002, work...","[go, arizona, auto, care, sinc, 2002, work, co..."
1,2L30O7G8IQ6HILpR0t5RFA,"part of a social event, we only had app's here...",5,0,0,0,2010-01-24,review,qiwajZigq_2twTmYofPmDQ,ST8Yzlk2MqKlcaLqL2djBg,415,part of a social event we only had apps here q...,"[part, of, a, social, event, we, only, had, ap...","[part, social, event, apps, quite, delicious, ...","[part, social, event, app, quit, delici, crab,..."
2,5h0EVAee-RDbbKfhd6FB0g,A great value for a far superior hair cut than...,5,2,1,0,2012-01-21,review,4VzaYvZntWBRbr8VmwCiWg,feQpvbp8jGBWMuG5uqbk3Q,284,A great value for a far superior hair cut than...,"[a, great, value, for, a, far, superior, hair,...","[great, value, far, superior, hair, cut, likel...","[great, valu, far, superior, hair, cut, like, ..."
3,6jV77Bs_Vu_rHkdUxu9JLQ,This review is for the JCPenny portrait studio...,3,1,2,1,2011-01-23,review,kwq3bK7BzPKLwXKqVRztHg,RwVaQNP1Ag-Seu3U9quGAg,1092,This review is for the JCPenny portrait studio...,"[this, review, is, for, the, jcpenny, portrait...","[review, jcpenny, portrait, studio, responsibl...","[review, jcpenni, portrait, studio, respons, c..."
4,8uV26l7-ktb4tZ5gRsIWWg,"The new Harkins Cine Capri, one of the first t...",5,0,1,0,2007-01-23,review,1pGC-0jvQ6vN5rzmCHjmfg,nWouNfZD3Pw08RYizxkqcA,355,The new Harkins Cine Capri one of the first th...,"[the, new, harkins, cine, capri, one, of, the,...","[new, harkins, cine, capri, one, first, things...","[new, harkin, cine, capri, one, first, thing, ..."


In [21]:
# Lemmatization
# import these modules 
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 

# Create an instance
wn = nltk.WordNetLemmatizer()

def lemmatizing(tokenized_text):

  text = [wn.lemmatize(word) for word in tokenized_text]
  return text

pandas_df['body_text_lemmatized'] = pandas_df['body_text_nostop'].apply(lambda x: lemmatizing(x))

pandas_df.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,review_id,review_text,stars,cool,useful,funny,review_date,review_type,business_id,user_id,length,body_text_clean,body_text_tokenized,body_text_nostop,body_text_stemmed,body_text_lemmatized
0,-7yxrdY13ay15rGB7WibMA,I have been going to Arizona Auto Care since a...,5,0,0,0,2010-01-16,review,Lh9nz0KYyzE-YRbKuCYeUw,ayKW9eWwGFcrtJaHcwZUCw,670,I have been going to Arizona Auto Care since a...,"[i, have, been, going, to, arizona, auto, care...","[going, arizona, auto, care, since, 2002, work...","[go, arizona, auto, care, sinc, 2002, work, co...","[going, arizona, auto, care, since, 2002, work..."
1,2L30O7G8IQ6HILpR0t5RFA,"part of a social event, we only had app's here...",5,0,0,0,2010-01-24,review,qiwajZigq_2twTmYofPmDQ,ST8Yzlk2MqKlcaLqL2djBg,415,part of a social event we only had apps here q...,"[part, of, a, social, event, we, only, had, ap...","[part, social, event, apps, quite, delicious, ...","[part, social, event, app, quit, delici, crab,...","[part, social, event, apps, quite, delicious, ..."
2,5h0EVAee-RDbbKfhd6FB0g,A great value for a far superior hair cut than...,5,2,1,0,2012-01-21,review,4VzaYvZntWBRbr8VmwCiWg,feQpvbp8jGBWMuG5uqbk3Q,284,A great value for a far superior hair cut than...,"[a, great, value, for, a, far, superior, hair,...","[great, value, far, superior, hair, cut, likel...","[great, valu, far, superior, hair, cut, like, ...","[great, value, far, superior, hair, cut, likel..."
3,6jV77Bs_Vu_rHkdUxu9JLQ,This review is for the JCPenny portrait studio...,3,1,2,1,2011-01-23,review,kwq3bK7BzPKLwXKqVRztHg,RwVaQNP1Ag-Seu3U9quGAg,1092,This review is for the JCPenny portrait studio...,"[this, review, is, for, the, jcpenny, portrait...","[review, jcpenny, portrait, studio, responsibl...","[review, jcpenni, portrait, studio, respons, c...","[review, jcpenny, portrait, studio, responsibl..."
4,8uV26l7-ktb4tZ5gRsIWWg,"The new Harkins Cine Capri, one of the first t...",5,0,1,0,2007-01-23,review,1pGC-0jvQ6vN5rzmCHjmfg,nWouNfZD3Pw08RYizxkqcA,355,The new Harkins Cine Capri one of the first th...,"[the, new, harkins, cine, capri, one, of, the,...","[new, harkins, cine, capri, one, first, things...","[new, harkin, cine, capri, one, first, thing, ...","[new, harkins, cine, capri, one, first, thing,..."


In [22]:
# Apply CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(pandas_df['review_text'])

In [23]:
# import pandas as pd

# X_counts_df = pd.DataFrame(X_counts.toarray(), columns=count_vect.get_feature_names())
# X_counts_df.head(10)

In [24]:
pandas_df_copy = pandas_df.copy()

pandas_df_copy = pandas_df_copy[['review_id', 'review_text', 'stars', 'cool', 'useful', 'funny', 'review_date', 'business_id', 'user_id', 'length', 'body_text_nostop', 'body_text_stemmed', 'body_text_lemmatized']]

# Convert pandas_df to sparks df
spark_df = spark.createDataFrame(pandas_df_copy)
spark_df.show(5)

+--------------------+--------------------+-----+----+------+-----+-----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+
|           review_id|         review_text|stars|cool|useful|funny|review_date|         business_id|             user_id|length|    body_text_nostop|   body_text_stemmed|body_text_lemmatized|
+--------------------+--------------------+-----+----+------+-----+-----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+
|-7yxrdY13ay15rGB7...|I have been going...|    5|   0|     0|    0| 2010-01-16|Lh9nz0KYyzE-YRbKu...|ayKW9eWwGFcrtJaHc...|   670|[going, arizona, ...|[go, arizona, aut...|[going, arizona, ...|
|2L30O7G8IQ6HILpR0...|part of a social ...|    5|   0|     0|    0| 2010-01-24|qiwajZigq_2twTmYo...|ST8Yzlk2MqKlcaLqL...|   415|[part, social, ev...|[part, social, ev...|[part, social, ev...|
|5h0EVAee-RDbbKfhd...|A great value for.

In [25]:
# Import functions
from pyspark.ml.feature import HashingTF, IDF, StringIndexer

In [26]:
# Make stars values a list
from pyspark.sql.functions import col, split
spark_df = spark_df.withColumn("star_array", split(col("stars"), " "))
spark_df.show()

+--------------------+--------------------+-----+----+------+-----+-----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+
|           review_id|         review_text|stars|cool|useful|funny|review_date|         business_id|             user_id|length|    body_text_nostop|   body_text_stemmed|body_text_lemmatized|star_array|
+--------------------+--------------------+-----+----+------+-----+-----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+
|-7yxrdY13ay15rGB7...|I have been going...|    5|   0|     0|    0| 2010-01-16|Lh9nz0KYyzE-YRbKu...|ayKW9eWwGFcrtJaHc...|   670|[going, arizona, ...|[go, arizona, aut...|[going, arizona, ...|       [5]|
|2L30O7G8IQ6HILpR0...|part of a social ...|    5|   0|     0|    0| 2010-01-24|qiwajZigq_2twTmYo...|ST8Yzlk2MqKlcaLqL...|   415|[part, social, ev...|[part, social, ev...|[part, social, ev.

In [27]:
# Initialize a CoutVectorizer
from pyspark.ml.feature import CountVectorizer
star_vectorizer = CountVectorizer(inputCol="star_array", outputCol="stars_one_hot", vocabSize=5, minDF=1.0)

In [28]:
# Create a vector model
star_vector_model = star_vectorizer.fit(spark_df)

In [29]:
# One hot encoded column
df_ohe = star_vector_model.transform(spark_df)
df_ohe.show(3)

+--------------------+--------------------+-----+----+------+-----+-----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+-------------+
|           review_id|         review_text|stars|cool|useful|funny|review_date|         business_id|             user_id|length|    body_text_nostop|   body_text_stemmed|body_text_lemmatized|star_array|stars_one_hot|
+--------------------+--------------------+-----+----+------+-----+-----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+-------------+
|-7yxrdY13ay15rGB7...|I have been going...|    5|   0|     0|    0| 2010-01-16|Lh9nz0KYyzE-YRbKu...|ayKW9eWwGFcrtJaHc...|   670|[going, arizona, ...|[go, arizona, aut...|[going, arizona, ...|       [5]|(3,[0],[1.0])|
|2L30O7G8IQ6HILpR0...|part of a social ...|    5|   0|     0|    0| 2010-01-24|qiwajZigq_2twTmYo...|ST8Yzlk2MqKlcaLqL...|   415|[par

In [30]:
# Create all the features to the data set
star_rating = StringIndexer(inputCol='stars',outputCol='label')
hashingTF = HashingTF(inputCol="body_text_stemmed", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [31]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
# Create feature vector 
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [32]:
# Create and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[star_rating, hashingTF, idf, clean_up])

In [33]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(df_ohe)
cleaned = cleaner.transform(df_ohe)

In [34]:
cleaned.show(5)

+--------------------+--------------------+-----+----+------+-----+-----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+-------------+-----+--------------------+--------------------+--------------------+
|           review_id|         review_text|stars|cool|useful|funny|review_date|         business_id|             user_id|length|    body_text_nostop|   body_text_stemmed|body_text_lemmatized|star_array|stars_one_hot|label|          hash_token|           idf_token|            features|
+--------------------+--------------------+-----+----+------+-----+-----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+-------------+-----+--------------------+--------------------+--------------------+
|-7yxrdY13ay15rGB7...|I have been going...|    5|   0|     0|    0| 2010-01-16|Lh9nz0KYyzE-YRbKu...|ayKW9eWwGFcrtJaHc...|   670|[going, arizon

<br></br>**Pipeline**<br></br>

In [35]:
# Import functions
# from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer

In [36]:
# Make stars values a list
# from pyspark.sql.functions import col, split
# spark_df = spark_df.withColumn("star_array", split(col("stars"), " "))
# spark_df.show()

In [37]:
# Initialize a CoutVectorizer
# from pyspark.ml.feature import CountVectorizer
# star_vectorizer = CountVectorizer(inputCol="star_array", outputCol="stars_one_hot", vocabSize=5, minDF=1.0)

In [38]:
# Create a vector model
# star_vector_model = star_vectorizer.fit(spark_df)

In [39]:
# # One hot encoded column
# df_ohe = star_vector_model.transform(spark_df)
# df_ohe.show(3)

In [40]:
# Create all the features to the data set
#star_rating = StringIndexer(inputCol='stars_one_hot',outputCol='label')
# tokenizer = Tokenizer(inputCol="review_text", outputCol="token_text")
# stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
# hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token')
# idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [41]:
# from pyspark.ml.feature import VectorAssembler
# from pyspark.ml.linalg import Vector
# # Create feature vector 
# clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [42]:
# Create and run a data processing Pipeline
# from pyspark.ml import Pipeline
# data_prep_pipeline = Pipeline(stages=[tokenizer, stopremove, hashingTF, idf, clean_up])

In [43]:
# Fit and transform the pipeline
# cleaner = data_prep_pipeline.fit(df_ohe)
# cleaned = cleaner.transform(df_ohe)

In [44]:
# cleaned.show()

In [45]:
# Drop redundant column
# x=cleaned.drop('review_type')

# **Naive Bayes**

In [46]:
#Drop intermediate columns
x=cleaned.select('features', 'label')
x.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(262145,[7327,133...|  0.0|
|(262145,[8804,130...|  0.0|
|(262145,[14934,30...|  0.0|
|(262145,[5173,844...|  1.0|
|(262145,[750,1184...|  0.0|
+--------------------+-----+
only showing top 5 rows



In [47]:
x.dtypes

[('features', 'vector'), ('label', 'double')]

In [48]:
# Import col
from pyspark.sql.functions import col

# Change column DataType for stars
x = x.withColumn('label', col('label').cast('int'))

x.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(262145,[7327,133...|    0|
|(262145,[8804,130...|    0|
|(262145,[14934,30...|    0|
|(262145,[5173,844...|    1|
|(262145,[750,1184...|    0|
+--------------------+-----+
only showing top 5 rows



In [49]:
# Rename class to label
# x = x.withColumnRenamed('stars_one_hot', 'label')
# x.show(5)

In [50]:
#import pandas as pd
# data_df = x.select('*').toPandas()
# x = spark.createDataFrame(data_df)

In [51]:
# Break data down into a training set and a testing set
training, testing = x.randomSplit([0.7, 0.3], 21)

**Naive Bayes**

In [52]:
from pyspark.ml.classification import NaiveBayes
# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [53]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(predictor.transform(testing))
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.522249


**Logistic Regression**

In [54]:
from pyspark.ml.classification import LogisticRegression
# Create a Naive Bayes model and fit training data
lg = LogisticRegression()
predictor = lg.fit(training)

In [55]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(predictor.transform(testing))
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.701011


**Multilayer Percepron**

In [56]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [57]:
# Split the data into train and test
splits = x.randomSplit([0.8, 0.2], 1234)
train = splits[0]
test = splits[1]

In [58]:
train.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(262145,[39,353,3...|    0|
|(262145,[47,1251,...|    1|
|(262145,[47,1511,...|    0|
|(262145,[47,2835,...|    1|
|(262145,[58,353,1...|    0|
|(262145,[67,1353,...|    0|
|(262145,[74,2835,...|    0|
|(262145,[74,11137...|    0|
|(262145,[90,4096,...|    0|
|(262145,[91,1176,...|    0|
|(262145,[97,1353,...|    1|
|(262145,[97,1810,...|    2|
|(262145,[97,5381,...|    0|
|(262145,[100,8804...|    1|
|(262145,[106,170,...|    0|
|(262145,[170,976,...|    1|
|(262145,[170,1176...|    1|
|(262145,[170,1353...|    0|
|(262145,[198,535,...|    0|
|(262145,[198,590,...|    0|
+--------------------+-----+
only showing top 20 rows



In [59]:
# specify layers for the neural network:
# input layer of size 3 (features), two intermediate of size 5 and 4
# and output of size 5 (classes)
layers = [262145, 131073, 3]

In [60]:
# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=10, layers=layers, blockSize=128, seed=1234)

In [61]:
# train the model
model = trainer.fit(train)

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 50226)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/content/spark-2.4.6-bin-hadoop2.7/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/content/spark-2.4.6-bin-hadoop2.7/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/content/spark-2.4.6-bin-hadoop2.7/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File

Py4JNetworkError: ignored

In [ ]:
# Authenticate user
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Set project id
project_id = 'xy-yelp'

In [ ]:
# Set project
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
# File path to save json file to
filepath = '/tmp/ml_j.json'

In [ ]:
# Save json file **** this will break if the file already exists, which it does at this point, therefore its commented out for now
x.coalesce(1).write.format('json').save(filepath)

In [ ]:
# Bucket name
bucket_name = 'xy-bucket'

In [ ]:
# Copy saved file from /tmp to bucket
!gsutil cp -r /tmp/ml_j.json/ gs://{bucket_name}/json_files/

Copying file:///tmp/ml_j.json/.part-00000-f8a8f21c-0ebe-434c-a8b3-a5c8988dd298-c000.json.crc [Content-Type=application/octet-stream]...
Copying file:///tmp/ml_j.json/._SUCCESS.crc [Content-Type=application/octet-stream]...
Copying file:///tmp/ml_j.json/_SUCCESS [Content-Type=application/octet-stream]...
Copying file:///tmp/ml_j.json/part-00000-f8a8f21c-0ebe-434c-a8b3-a5c8988dd298-c000.json [Content-Type=application/json]...
\
Operation completed over 4 objects/63.0 MiB.                                     
